We're almost there:
- We loaded the documents
- We've split them in to relevant parts (splitters)
- We've had the vector database calculate the embeddings and store the related parts

Related Langchain example: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
%pip install -q langchain langchain-openai langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

We connect to the previously created vector database

In [3]:

# Set the embeddings function
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

# Create a chromadb client
import chromadb

collection_name="my_langchain"
chroma_client = chromadb.PersistentClient(path="./chromadb")
from langchain_community.vectorstores import Chroma
vectorstore = Chroma(embedding_function=embeddings_model,client=chroma_client,collection_name=collection_name)


Now this is the step we put it all together by using the RetrievalQA chain.
This combines the vectordatabase , the prompt , the llm to return answer.


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
from _lessonshelper.pretty_print_callback_handler import PrettyPrintCallbackHandler

pretty_print_callback = PrettyPrintCallbackHandler()
llm.callbacks = [pretty_print_callback]

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), return_source_documents=True
)


For comparison let's first do our query through the regular llm call.
You see it returns 4 authors of the Devops Handbook.

In [5]:

query = "Who were the people involved in writing the devops Handbook ? If there are multiple authors return them all. Return the result as json and use the field firstname and lastname"


# First via the llm
from langchain.schema import (
    HumanMessage,
)

answer = llm.invoke([HumanMessage(content=query)])
from pprint import pprint
pprint(answer)


[chat_model][start] - prompts : Who were the people involved in writing the devops Handbook ? If there are multiple authors return them all. Return the result as json and use the field firstname and lastname

[llm][end] - generation The authors of "The DevOps Handbook" are Gene Kim, Jez Humble, Patrick Debois, and John Willis. Here is the information in JSON format:

[
  {
    "firstname": "Gene",
    "lastname": "Kim"
  },
  {
    "firstname": "Jez",
    "lastname": "Humble"
  },
  {
    "firstname": "Patrick",
    "lastname": "Debois"
  },
  {
    "firstname": "John",
    "lastname": "Willis"
  }
]
AIMessage(content='The authors of "The DevOps Handbook" are Gene Kim, Jez Humble, Patrick Debois, and John Willis. Here is the information in JSON format:\n\n[\n  {\n    "firstname": "Gene",\n    "lastname": "Kim"\n  },\n  {\n    "firstname": "Jez",\n    "lastname": "Humble"\n  },\n  {\n    "firstname": "Patrick",\n    "lastname": "Debois"\n  },\n  {\n    "firstname": "John",\n    "lastnam

Now with the retrieval chain we see it returns 5 authors. The extra one being John Allspaw who wrote the foreword.

In [6]:

# next via the llm via Retrieval Augmented Generation
answer = qa_chain.invoke({"query": query})
pprint(answer)


[chat_model][start] - prompts : Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The Devops Handbook was written by the following authors: Gene Kim, Jez Humble , John Willis , Patrick Debois and John Allspaw.  
Gene Kim is a multiple award-winning entrepreneur, the founder and former CTO of Tripwire and a researcher. He is passionate about IT operations, security and compliance, and how IT organizations successfully transform from "good to great." He lives in Portland, Oregon.  
Jez Humble is an award-winning author and researcher on software who has spent his career tinkering with code, infrastructure, and product development in organizations of varying sizes across three continents. He works at 18F, teaches at UC Berkeley, and is co-founder of DevOps Research and Assessment LLC.  
Patrick Debois is an independent IT-consultant who is bridging the gap betwee

This combination of RAG is one of the primary patterns people are using LLMs with their own data.